In [28]:
### import software packages that we'll use
import os
import pickle
import numpy as np
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
from torchsummary import summary
import wandb

ModuleNotFoundError: No module named 'torchsummary'

In [2]:
### get data path
### the data is organized as follows: we have peripheral blood mononuclear cell (PBMC) samples for 9 patients,
### meaning 9 separate slides were imaged with immunofluorescence microscopy. each image is a 3-channel image,
### where each channel captures a different antibody. the antibodies used in this experiment are DAPI,
### which marks the nucleus of the cell, CD45, which marks the cell membrane of immune cells, and PanCK, which
### marks the cell membrane of tumor cells. PanCK and CD45 and known to be mutually exclusive. each slide is split into 2+ regions
### of interest (ROIs), which are called scenes. scene 1 is the control well, meaning the cells were not stained
### with the immunofluorescent-tagged antibodies. this scene is not relevant to us. we have 15 total scenes. next,
### we ran our image analysis pipeline on each remaining scene to detect potential circulating hybrid cells (CHCs).

In [3]:
### define dataloader
### this loads the data into batches for training
data_path = 'dataset.pkl'
with open(data_path, 'rb') as handle:
    data = pickle.load(handle)

data.drop(labels='cell',axis=1,inplace=True)
data = data.to_numpy()

In [4]:
### define model
### this is the meat and potatoes
class Model(nn.Module):
     
    def __init__(self, input_size):
        super().__init__()
        
        #layers:
        self.l1 = nn.Linear(input_size, 16) # layer 1
        self.l2 = nn.Linear(16, 1)
        
        #activation functions:
        self.relu = nn.ReLU() #rectified linear activation function, introduces nonlinearity into the model
        self.sigmoid = nn.Sigmoid() #sigmoid activation for binary classification (transforms to 0 or 1)
    
    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.sigmoid(x)
        return x

In [20]:
class SingleCellClassifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = self.model(x)
        loss = F.binary_cross_entropy(x, y.reshape(-1,1).float())
        self.log("train_loss",loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        x = self.model(x)
        loss = F.binary_cross_entropy(x, y.reshape(-1,1).float())
        self.log("val_loss",loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [21]:
class PBMCDataset(Dataset):
    def __init__(self, data):
        self.embeddings = data[:,0]
        self.labels = data[:,1]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        embedding = self.embeddings[idx]
        label = self.labels[idx]
        
        return torch.tensor(embedding), torch.tensor(label)

In [22]:
train_set, val_set = torch.utils.data.random_split(PBMCDataset(data), [2000,834])

In [25]:
wandb_logger = WandbLogger(project='classifier')

wandb: Currently logged in as: heussner (changlab). Use `wandb login --relogin` to force relogin


In [26]:
train_loader = DataLoader(train_set)
valid_loader = DataLoader(val_set)

classifier = SingleCellClassifier(Model(input_size=64))

# train with both splits
trainer = Trainer(logger=wandb_logger)
trainer.fit(classifier, train_loader, valid_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/users/heussner/.local/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4]

  | Name  | Type  | Params
--------------------------------
0 | model | Model | 1.1 K 
--------------------------------
1.1 K     Trainable params
0         Non-trainable params
1.1 K     Total params
0.004     

Sanity Checking DataLoader 0:   0%|                                              | 0/2 [00:00<?, ?it/s]

/home/users/heussner/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/users/heussner/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|████████████████████████████████████████| 2000/2000 [00:06<00:00, 297.61it/s, v_num=cy1t]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   9%|███▍                                   | 73/834 [00:00<00:01, 385.79it/s]


...

Validation DataLoader 0:  82%|███████████████████████████████       | 681/834 [00:01<00:00, 404.11it/s]


Validation DataLoader 0:  90%|██████████████████████████████████▎   | 753/834 [00:01<00:00, 396.92it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 35:  62%|████████████████████████▎              | 1244/2000 [00:03<00:02, 345.65it/s, v_num=cy1t]

/home/users/heussner/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [27]:
summary(Model((64)))

NameError: name 'summary' is not defined